# Conversor de Voz

Utilice el siguiente código para convertir una forma de onda de voz mono de 16 kHz en otra.

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForSpeechToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate
example_speech = dataset[0]["audio"]["array"]

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_vc")
model = SpeechT5ForSpeechToSpeech.from_pretrained("microsoft/speecht5_vc")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

inputs = processor(audio=example_speech, sampling_rate=sampling_rate, return_tensors="pt")

# load xvector containing speaker's voice characteristics from a file
speaker_embeddings = torch.zeros((1, 512))  # or load xvectors from a file

speech = model.generate_speech(inputs["input_values"], speaker_embeddings, vocoder=vocoder)

import soundfile as sf
sf.write("speech.wav", speech.numpy(), samplerate=16000)

Ahora extraemos las caracteristicas de una voz diferente

In [ ]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")
signal, fs =torchaudio.load('tests/samples/ASR/spk1_snt1.wav')
embeddings = classifier.encode_batch(signal)
# Podemos guardarlas y cargarlas luego con:
# speaker_embeddings = np.load("xvector_speaker_embedding.npy")
# speaker_embeddings = torch.tensor(speaker_embeddings).unsqueeze(0)

speech = model.generate_speech(inputs["input_values"], speaker_embeddings, vocoder=vocoder)

import soundfile as sf
sf.write("speech.wav", speech.numpy(), samplerate=16000)

Ahora, analizemos el interior de este modelo

## Ahora utilizaremos un conversor voz, basado en TTS

Primero, instalamos la libreria necesaria.

In [ ]:
%pip install TTS

Utilice el siguiente código para convertir un texto a una forma de onda de voz mono de 16 kHz.

In [ ]:
from huggingface_hub import snapshot_download
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
import os

CACHE_MODEL_DIR = "./models"

model_path = snapshot_download(
    repo_id="coqui/XTTS-v2",
    cache_dir=CACHE_MODEL_DIR
)

config = XttsConfig()
config.load_json(os.path.join(model_path, "config.json"))
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_dir=model_path, eval=True)
model.cuda()

mensaje = "Me llevo un tiempo desarrollar una voz y ahora que la tengo no pienso quedarme callada"

speaker_wav = "audio_loc.wav"
outputs = model.synthesize(
    mensaje,
    config,
    speaker_wav=speaker_wav,
    gpt_cond_len=3,
    language="es",
)

Audio(outputs["wav"], rate=24000)

Ahora, analizemos el interior de este modelo